In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from csv import reader
from pyspark.sql.types import DoubleType,StringType,IntegerType,DateType
from pyspark.sql.functions import col,when,count,desc
from pyspark.sql.functions import format_string
import sys
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

In [2]:
sc=SparkContext.getOrCreate()
spark = SparkSession(sc)
df=spark.read.format("csv").option("header", "true").load("/FileStore/tables//matches_2019.csv")
df.columns



Out[2]: ['id',
 'season',
 'city',
 'date',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'result',
 'dl_applied',
 'winner',
 'win_by_runs',
 'win_by_wickets',
 'player_of_match',
 'venue',
 'umpire1',
 'umpire2',
 'umpire3']

In [3]:
df=df.withColumn("win_by_runs", df["win_by_runs"].cast(IntegerType()))
df=df.withColumn("win_by_wickets", df["win_by_wickets"].cast(IntegerType()))
df=df.withColumn("season", df["season"].cast(DateType()))
df.select("toss_decision").distinct().show()

+-------------+
toss_decision|
+-------------+
 field|
 bat|
+-------------+

In [4]:
!pip install altair
import altair as alt

In [5]:
teamstatistics=df.select(["season","result"])
teamsresult=teamstatistics.groupBy(["season"]).count()
teamsresultseason=teamsresult.toPandas()
teamsresultseason['season'] = pd.to_datetime(teamsresultseason['season']) 
teamsresultseason["year"] = teamsresultseason.season.dt.year
alt.Chart(teamsresultseason).mark_line().encode(
    alt.X('year:N'),
    alt.Y('count:Q'),
   
).properties(
    width=500,
    height=300,
    title='# matches by year'
)
# display(teamsresult)

In [7]:
teamsresultseason=teamstatistics.groupBy(["season","result"]).count()
teamsresultseason=teamsresultseason.toPandas()
teamsresultseason['season'] = pd.to_datetime(teamsresultseason['season']) 
teamsresultseason["year"] = teamsresultseason.season.dt.year
alt.Chart(teamsresultseason).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('result:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Seasons vs results'
)


In [8]:

results=df.select(["result","winner"])
results=results.groupBy(["winner"]).count()
resultsdf=results.toPandas()
resultsdf

bars = alt.Chart().mark_bar().encode(
    x=alt.X('winner:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count:Q'),
    #color='borough'
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='count:Q'
)

alt.layer(bars, texts, data=resultsdf).properties(
    width=800,
    height=400
)
# display(results.orderBy("count"))

In [9]:
results=df.select(["season","winner"])
results=results.groupBy(["season","winner"]).count()
resultsdf=results.toPandas()
resultsdf['season'] = pd.to_datetime(resultsdf['season']) 
resultsdf["year"] = resultsdf.season.dt.year
alt.Chart(resultsdf).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('winner:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Seasons vs results'
)

In [10]:
toss=df.filter(df.toss_winner==df.winner)
results=toss.select(["season","winner"])
results=results.groupBy(["season","winner"]).count()
resultsdf=results.toPandas()
resultsdf['season'] = pd.to_datetime(resultsdf['season']) 
resultsdf["year"] = resultsdf.season.dt.year
win=alt.Chart(resultsdf).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('winner:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Won Toss and Won Match'
)


toss1=df.filter(df.toss_winner!=df.winner)
results1=toss1.select(["season","winner"])
results1=results1.groupBy(["season","winner"]).count()
resultsdf1=results1.toPandas()
resultsdf1['season'] = pd.to_datetime(resultsdf1['season']) 
resultsdf1["year"] = resultsdf1.season.dt.year
loss=alt.Chart(resultsdf1).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('winner:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Won Toss But Lost Match'
)

win | loss

In [11]:
toss=df.filter((df.toss_winner==df.winner) & (df.toss_decision=="bat") )
results=toss.select(["season","winner"])
results=results.groupBy(["season","winner"]).count()
resultsdf=results.toPandas()
resultsdf['season'] = pd.to_datetime(resultsdf['season']) 
resultsdf["year"] = resultsdf.season.dt.year
win=alt.Chart(resultsdf).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('winner:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Won Toss, elected to bat and Won Match'
)


toss1=df.filter((df.toss_winner!=df.winner) & (df.toss_decision!="bat") )
results1=toss1.select(["season","winner"])
results1=results1.groupBy(["season","winner"]).count()
resultsdf1=results1.toPandas()
resultsdf1['season'] = pd.to_datetime(resultsdf1['season']) 
resultsdf1["year"] = resultsdf1.season.dt.year
loss=alt.Chart(resultsdf1).mark_rect().encode(
    alt.X('year:N'),
    alt.Y('winner:N'),
    alt.Color('count')
).properties(
    width=400,
    height=400,
    title='Won Toss,elected to field But Lost Match'
)

win | loss

In [12]:
results =df.groupBy(["team1","team2"]).count() 
sourcedf=results.toPandas()
alt.Chart(sourcedf).mark_bar().encode(
    alt.Y('team1:O',sort="-x"),
    alt.X('count:Q'),
    facet=alt.Facet('team2:N', columns=5)
).properties(
    width=200,
    height=200,
    title='No.of match between team1 and team2'
  
)

In [13]:
results=results.
results =df.groupBy(["win_by_runs","season","winner"]).mean() 

sourcedf=results.toPandas()
# sourcedf=sourcedf[sourcedf["season"]!="null"]

sourcedf['season'] = pd.to_datetime(sourcedf['season']) 
sourcedf["year"] = resultsdf1.season.dt.year
alt.Chart(sourcedf).mark_bar().encode(
    alt.Y('winner:O',sort="-x"),
    alt.X('avg(win_by_runs):Q'),
    
    facet=alt.Facet('year:N', columns=5)
).properties(
    width=200,
    height=200,
    title='No.of match between team1 and team2'
  
)

In [14]:
results.select("season").distinct().show()

+----------+
 season|
+----------+
2009-01-01|
2016-01-01|
2011-01-01|
2008-01-01|
2017-01-01|
2010-01-01|
2013-01-01|
2019-01-01|
2012-01-01|
2018-01-01|
2015-01-01|
2014-01-01|
+----------+